In [1]:
from base.dataset import DataSet
from base.plotting import plot_long_tail, plot_sample_recall_precision, plot_recall_precision, plot_together
import os

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

## Dataset building from transactions
We will explore a dataset of a multi-purpose online store, providing us with a history of user-item interactions: views, purchases, save-to-carts...

The objective is to experiment with a few models to see their benefits and drawbacks, and how to use those to specialize in certain tasks.

In [ ]:
os.chdir('..')
dataset = DataSet.from_pickle(dataset_size='full')

Our dataset shows the typical case when working with recommender systems: **data sparcity** and **data imbalance**. When building a matrix of user-item interaction, it turns out to be _very very sparse_: users interact with relatively few items overall (data sparcity); meanwhile, popular items are few in quantity but frequent in activity, while most items only get interactions from a select group of users (data imbalance).

This leads us to the (relatively obvious) conclusion that focusing on just recommending popular items will not take us very far.

In [11]:
sales_count = plot_long_tail(dataset.metrics, metric='sales_count')
items_per_user = plot_long_tail(dataset.users, metric='n_relevant')
plot_together(sales_count, items_per_user)

    'data': [{'fillpattern': {'shape': ''},
              'hovertemplate': 'prod…

### Test-train separation

In this context, it is not straight-forward how the separation into test and train data should be done: given the importance of **continuity** and the interconnected nature of our dataset, it would be counter-productive to randomly divide our data.

Different strategies are commonplace within the community, but the one we will use here is **user tail-withholding**: we exclude a percentage (train-test split) of the relevant items for each user, withholding that information from the model training. When it comes time to validate our model, we will measure "accuracy" (however that is defined for recommendation systems) three times: one for the training set, one for the validation set, and one for the full dataset. This way we can see how much our model "memorized" and how much it "generalized".

## Measuring success

For traditional binary classifiers, `precision = correct positive / predicted positive` (how often our positive predictions are accurate) and `recall = correct positive / actual positives` (how often we detect real positives).

However, the power of recommender systems lies in that they are not "hit-or-miss" by only making one prediction. Instead, most frequently they return a **list** of several potential recommendations, probably ordered by likelihood of success. Therefore, accuracy in recommendations should take into account all given predictions, whether they were actually relevant or not, and how high in our list they were.

Generalizing to several predictions, we can extend their definitions as follows. Assume we make `N` sorted recommendations, and let `0 < k <= N` be a positive number. Then, we define:
- **Precision at k**: `P@k = relevant recommendations / k`, how many of my first `k` recommendations were relevant. Also called the **hit ratio at k** (`HR@k`).
- **Recall at k**: `R@k = relevant recommendations / all relevant items`, how many of the possible relevant items we ended up recommending within our first `k` suggestions.
- **Average Precision at k**: `AP@k = (P@1 * is_relevant(1) + ... + P@k * is_relevant(k)) / all relevant items`, a more subtle metric:
    - `0 <= AP@k <= 1`, with a value of zero if none of the `k` recommendations were relevant, and with a value of one if `k = all relevant items` and all recommendations were relevant.
    - `AP@k` is an increasing function of `k`, strictly so at `k` if the `k`-th recommendation is relevant and remaining constant otherwise.
    - For a fixed `k`, it reaches a maximum value if all predictions are relevant.
    - For a fixed `k` and same number of relevant predictions, `AP@k` is higher the earlier the relevant predictions are. If all relevant suggestions are at the beginning, then `AP@k = R@k`.
    - Another formulation of `AP@k` reveals how it is an amalgamation of precision, recall and order of success: `AP@k = P@1 * (R@1 - R@0) + ... + P@k * (R@k - R@(k-1))` (where `R@0 = 0`).
    - Variations of the formula replace `all relevant items` with `min(all relevant items, k)`, specially in cases where the number of recommendations given cannot be fairly compared against all possible relevant items. In any case, it only results in a scaling of `AP@k`.
- **Mean Average Precision at k**: `MAP@k = (AP@k(u_1) + ... + AP@k(u_r)) / all users`, the mean over all users/recommendation batches of average precisions. This lets us measure the accuracy of our model by using only one metric, independently of the number of users.
- **Rank at k**: the position of the first relevant suggestion when only considering the first `k`, 0 if none are.
- **Reciprocal rank at k**: the inverse of **Rank at k**, 0 if undefined. Useful for calculating the **Mean Reciprocal Rank at k** of a model (`MRR@k`).
- **Coverage**: `coverage = unique items ever recommended / all existing items`, a helpful measure to identify models that tend to default to only recommend "popular" and therefore "safe" items.

For example, assume we make `N = 3` predictions, and there were a total of 5 possible relevant items for our given prompt. These are all possible outcomes:

![](res/images/example_success_measures.png)

By fixing a single recommendation batch and creating the parametrized curve `(R@k, P@k)`, we can see it always moves to the right (recall either increases or stays the same the more you recommend), but it can bounce up and down a lot (precision increases if you make a relevant suggestion, decreases otherwise):

In [ ]:
plot_sample_recall_precision(n_recommendations=100)

In [ ]:
anchor_user = dataset.get_random_user()
anchor_item = dataset.get_random_product_from_user(anchor_user.name)
print(f'Making recommendations for user {anchor_user.name}, with {anchor_user["relevant_items"]} relevant items.')
print(f'Chose item {anchor_item} as recommender prompt.')

In [ ]:
user_relevant_items = dataset.products.loc[anchor_user['relevant_items']]
user_relevant_items

## Model testing

### Random recommender

We just take a selection at random from all possible products. We do not expect this model to get a match _ever_, but it is still fun to try.

In [ ]:
from models.random_model import RandomRecommender
random_model = RandomRecommender(
    dataset=dataset
)
results, model_info = random_model.recommend(user=anchor_user.name, item=anchor_item)
model_info

In [ ]:
results.matches

In [10]:
random_model.evaluate_performance()
random_model.evaluate_accuracy(10)

100%|██████████| 100/100 [00:05<00:00, 19.77it/s]


------------------------------------
Random Recommender model - Performance
------------------------------------
Model setup time: 0.000s
Average time: 0.002s
Worst time: 0.002s
Best time: 0.002s


100%|██████████| 100/100 [00:04<00:00, 22.90it/s]

------------------------------------
Random Recommender model - Validation statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.0000 | 0.0000 |  0.0000 | 0.0000 |
|     R@10     |  0.0000 | 0.0000 |  0.0000 | 0.0000 |
| P@10 (HR@10) |  0.00%  | 0.00%  |  0.00%  | 0.00%  |
|   Rank@10    |   nan   |  nan   |   nan   |  nan   |
|  RecRank@10  |  0.0000 | 0.0000 |  0.0000 | 0.0000 |
+--------------+---------+--------+---------+--------+
------------------------------------
Random Recommender model - Full statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.0000 | 0.0000 |  0.0000 | 0.0000 |
|     R@10     |  0.0000 | 0.0000 |  0.0000 |


/Users/josean/.conda/envs/recommender-systems/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

/Users/josean/.conda/envs/recommender-systems/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice



### Baseline recommender

First approach at taking into consideration the interactions between products, by building a **co-occurrence** matrix: element `M(i,j)` contains the number of times products `i` and `j` are bought by the same person.

Then, whenever we need to look for recommendations prompted by item `r`, we look at the row `M(r,·)` (or column `M(·,r)`) and get the `k` items with highest sales count.

This model essentially recommends popular items (and therefore has all limitations attached to that method), but at least "popularity" is defined in an aggregated sense.

In [11]:
from models.baseline_model import BaselineRecommender
baseline_model = BaselineRecommender(
    dataset=dataset
)

Building co-occurrence matrix...
Done


This model gives us a good way to verify that in most cases (even more in real-world scenarios), the density of the relationship matrix (either item co-occurrence or user-item interaction) is very low:

In [ ]:
baseline_model.matrix_density

In [ ]:
results, model_info = baseline_model.recommend(user=anchor_user.name, item=anchor_item)
model_info

,category_id,category_code,brand,price,category_code_L1,category_code_L2,category_code_L3,category_code_L4,co-popularity
P-1004873,2053013555631882655,electronics.smartphone,samsung,411.57,electronics,smartphone,None,None,734.0
P-1004767,2053013555631882655,electronics.smartphone,samsung,277.97,electronics,smartphone,None,None,381.0
P-1004870,2053013555631882655,electronics.smartphone,samsung,329.46,electronics,smartphone,None,None,332.0
P-1004856,2053013555631882655,electronics.smartphone,samsung,136.17,electronics,smartphone,None,None,199.0
P-1004874,2053013555631882655,electronics.smartphone,samsung,385.75,electronics,smartphone,None,None,193.0
P-1004872,2053013555631882655,electronics.smartphone,samsung,334.37,electronics,smartphone,None,None,164.0
P-1004768,2053013555631882655,electronics.smartphone,samsung,277.97,electronics,smartphone,None,None,141.0
P-1005115,2053013555631882655,electronics.smartphone,apple,1015.70,electronics,smartphone,None,None,133.0
P-1005014,2053013555631882655,electronics.smartphone,samsung,614.55,electronics,smartphone,None,None,130.0
P-1002544,2053013555631882655,electronics.smartphone,apple,499.35,electronics,smartphone,None,None,114.0


In [ ]:
results.matches

0     True
1     True
2     True
3     True
4    False
5     True
6    False
7     True
8     True
9     True
Name: matches, dtype: bool

In [ ]:
baseline_model.evaluate_performance()
baseline_model.evaluate_accuracy(10)

100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


------------------------------------
Baseline Recommender model - Performance
------------------------------------
Model setup time: 53.288s
Average time: 0.612s
Worst time: 0.643s
Best time: 0.598s


100%|██████████| 100/100 [01:05<00:00,  1.52it/s]

------------------------------------
Baseline Recommender model - Validation statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.0785 | 0.0000 |  0.6429 | 0.0000 |
|     R@10     |  0.1728 | 0.0000 |  1.0000 | 0.0000 |
| P@10 (HR@10) |  4.10%  | 0.00%  |  30.00% | 0.00%  |
|   Rank@10    |  3.3030 | 3.0000 |  8.0000 | 1.0000 |
|  RecRank@10  |  0.1540 | 0.0000 |  1.0000 | 0.0000 |
+--------------+---------+--------+---------+--------+
------------------------------------
Baseline Recommender model - Full statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.1374 | 0.0742 |  0.8521 | 0.0000 |
|     R@10     |  0.1716 | 0.1333 |  0.75

### Ad-hoc recommender
We use simple manual rules to determine the similarity score between two products.

In this particular example, we set the rules as follows:
- **Compute category proximity**: a percentage of how much the category of two products match. For example, two products from the same category have a score of 1, while one from `electronics.smartphones` and another from `electronics.headphones` have a score of 0.5. Keep only those with a score of at least 0.5 (parametrizable).
- **Brand score**: 1 if they match, 0 otherwise. Scaled by a parameter.
- **Popularity within category**: from selection, get percentile of each product when ordering by times sold. Scaled by a parameter.
- **Cheapest first**: when two candidates have the same score, the cheapest gets priority.

In [ ]:
from models.ad_hoc_model import AdHocRecommender
ad_hoc_model = AdHocRecommender(
    dataset=dataset
)

In [ ]:
results, model_info = ad_hoc_model.recommend(user=anchor_user.name, item=anchor_item)
model_info

,brand,price,category_code_L1,category_code_L2,category_code_L3,category_code_L4,category_score,brand_score,popularity_score,score
product_id,,,,,,,,,,
P-1004856,samsung,136.17,electronics,smartphone,None,None,1.0,True,0.067259,0.067726
P-1004767,samsung,277.97,electronics,smartphone,None,None,1.0,True,0.048765,0.049241
P-1005115,apple,1015.70,electronics,smartphone,None,None,1.0,False,0.038191,0.038171
P-1004833,samsung,180.16,electronics,smartphone,None,None,1.0,True,0.028745,0.029230
P-1002544,apple,499.35,electronics,smartphone,None,None,1.0,False,0.024402,0.024389
P-1004870,samsung,329.46,electronics,smartphone,None,None,1.0,True,0.023371,0.023859
P-1004249,apple,849.16,electronics,smartphone,None,None,1.0,False,0.019729,0.019719
P-1004836,samsung,254.81,electronics,smartphone,None,None,1.0,True,0.017070,0.017562
P-1005105,apple,1450.72,electronics,smartphone,None,None,1.0,False,0.017320,0.017311


In [ ]:
results.matches

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: matches, dtype: bool

In [ ]:
ad_hoc_model.evaluate_performance()
ad_hoc_model.evaluate_accuracy(10)

100%|██████████| 100/100 [00:08<00:00, 11.15it/s]


------------------------------------
Ad-Hoc Recommender model - Performance
------------------------------------
Model setup time: 0.000s
Average time: 0.049s
Worst time: 0.054s
Best time: 0.037s


100%|██████████| 100/100 [00:09<00:00, 11.08it/s]

------------------------------------
Ad-Hoc Recommender model - Validation statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.0699 | 0.0000 |  0.7500 | 0.0000 |
|     R@10     |  0.1588 | 0.0000 |  1.0000 | 0.0000 |
| P@10 (HR@10) |  4.00%  | 0.00%  |  20.00% | 0.00%  |
|   Rank@10    |  3.8750 | 3.0000 | 10.0000 | 1.0000 |
|  RecRank@10  |  0.1456 | 0.0000 |  1.0000 | 0.0000 |
+--------------+---------+--------+---------+--------+
------------------------------------
Ad-Hoc Recommender model - Full statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.1008 | 0.0500 |  0.5583 | 0.0000 |
|     R@10     |  0.1294 | 0.1000 |  0.5000 |

### Word2Vec model

We leverage tools developed originally for NLP tasks, such as word vectorization embeddings, to help us detect patterns in items bought together, instead of relying purely on buy count.

For this, we apply the Word2Vec model: it is a shallow neural network (only with 1 layer) used to build embeddings of the one-hot encoding of a "vocabulary" into a much smaller dimensional space, using a corpus of "sentences" (order lists of "words") as a guide for how often words appear next to each other.

Two possible modalities exist for the Word2Vec algorithm: the Skip-Gram (SG) and the Continous-Bag-of-Words (CBOW). Both are useful to our particular case, but we will be using the first one. To put it simply,
- **Skip-Gram** takes one word as input, and outputs the probability of each word being within its window.
- **CBOW** takes a window without the middle word, and outputs the probability of each word being the missing word.

More information can be found in this fantastic article by [FastForwardLabs](https://session-based-recommenders.fastforwardlabs.com/), from which we took this infographic:
![](images/word2vec.png)

In [ ]:
from models.word2vec_model import Word2VecRecommender
word2vec_model = Word2VecRecommender(
    dataset=dataset
)

Extracting user sessions...
Training model...
Done!


In [ ]:
results, model_info = word2vec_model.recommend(user=anchor_user.name, item=anchor_item)
model_info

,brand,price,category_code_L1,category_code_L2,category_code_L3,category_code_L4
product_id,,,,,,
P-2800590,dauscher,347.24,appliances,kitchen,refrigerators,None
P-28716958,respect,118.15,apparel,shoes,None,None
P-12300842,crown,43.50,construction,tools,drill,None
P-1003894,lg,408.37,electronics,smartphone,None,None
P-28716953,respect,118.15,apparel,shoes,None,None
P-1004528,samsung,506.26,electronics,smartphone,None,None
P-1307443,asus,545.68,computers,notebook,None,None
P-100012188,respect,66.41,apparel,shoes,None,None
P-28716954,respect,118.15,apparel,shoes,None,None


In [ ]:
results.matches

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: matches, dtype: bool

In [ ]:
word2vec_model.evaluate_performance()
word2vec_model.evaluate_accuracy(10)

100%|██████████| 100/100 [00:08<00:00, 11.18it/s]


------------------------------------
Ad-Hoc Recommender model - Performance
------------------------------------
Model setup time: 0.000s
Average time: 0.048s
Worst time: 0.053s
Best time: 0.036s


100%|██████████| 100/100 [00:09<00:00, 11.03it/s]

------------------------------------
Ad-Hoc Recommender model - Validation statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.0745 | 0.0000 |  1.0000 | 0.0000 |
|     R@10     |  0.1935 | 0.0000 |  1.0000 | 0.0000 |
| P@10 (HR@10) |  4.80%  | 0.00%  |  30.00% | 0.00%  |
|   Rank@10    |  4.1389 | 3.5000 | 10.0000 | 1.0000 |
|  RecRank@10  |  0.1406 | 0.0000 |  1.0000 | 0.0000 |
+--------------+---------+--------+---------+--------+
------------------------------------
Ad-Hoc Recommender model - Full statistics
------------------------------------
+--------------+---------+--------+---------+--------+
|    Metric    | Average | Median | Highest | Lowest |
+--------------+---------+--------+---------+--------+
|    MAP@10    |  0.1194 | 0.0542 |  0.7161 | 0.0000 |
|     R@10     |  0.1525 | 0.1000 |  0.6667 |

### Matrix Factorization model

One of the first models to pop up in the context of recommendation systems was [FunkSVD](https://github.com/gbolmier/funk-svd), an algorithm based on a factorization of the **user-item rating** matrix.

Let's assume that `n = number of users`, `m = number of items`, and fix a natural number `1 <= k`. The *rating matrix* `R` is constructed so that `R(i,j)` is the "rating" that user `i` gave to item `j` (this can be an explicit, fixed-range rating, or some sort of implicit feedback for the interaction, positive, negative...), of size `n x m`. By desing, `R` is a **sparse** matrix that we want to fill in. To do this, we attempt to factor the matrix `R` into smaller, lower-rank matrices. **FunkSVD** applies singular value decomposition to `R`, and fills missing values in the matrix by carrying out the product of the decomposition.

Our MF model will try to factor `R` into two matrices, `U` and `P`, of sizes `n x k` and `k x m`. Here, `k` is the dimension of the **latent feature space**: both `U` and `P` work as embeddings of users and products into a common, "latent feature" lower-dimensional space, where each component represents a concept or "latent feature".

![](res/images/mf.png)

We can understand this latent features as `k` inferred concepts by the model, so that `U` (`P`) transforms each user (item) into its "affinity" to each concept. Then, to get the likelihood that user `i` likes item `j`, we just take the dot product of `U_i` and `P_j`.

![](res/images/latent_features.png)